In [7]:
from dotenv import load_dotenv
load_dotenv(r'.env')
import os
temp = os.environ.get('COMET_API_KEY')

In [6]:
# Importer comet_ml en haut de votre fichier, avant sklearn !
from comet_ml import Experiment

# Créer une expérience avec votre clé api
exp = Experiment(
    api_key=os.environ.get(temp),#ne pas coder en dur!
    project_name='milestone2',
    workspace='ift6758-a02',
)
# ... Faire de la science des données ...
exp.log_metrics({"auc": 12, "acc": 0.8, "loss": 0.65})


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/ift6758-a02/milestone2/9698eb944b8f4c95ae2ee66f31abf8db
COMET INFO:   Metrics:
COMET INFO:     acc  : 0.8
COMET INFO:     auc  : 12
COMET INFO:     loss : 0.65
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     git metadata        : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is l

COMET WARNING: Unknown error exporting current conda environment
COMET WARNING: Unknown error retrieving Conda package as an explicit file
COMET WARNING: Unknown error retrieving Conda information
